In [1]:
import pandas as pd
import numpy as np
import json
import string

# 筛选预测的

In [77]:
pat = json.load(open("output/prediction_extend.json", "r"))
pats = {}
for spec_type in pat:
    for spec in pat[spec_type]:
        vbp = {}
        for paper_id, paper in pat[spec_type][spec].items():
            vbp[paper_id+":"+spec] = paper
            vbp[paper_id+":"+spec]["spec_type"] = spec_type
            vbp[paper_id+":"+spec]["species"] = spec
        pats.update(vbp)
        
with open("output/prediction_extend_list.json","w") as f:
    json.dump(pats,f)

# 合并机器学习数据

In [2]:
with open("output/prediction_extend_list.json","r") as f:
    pats = json.load(f)
pat_df = pd.DataFrame(pats.values(), index=pats.keys())

evi_df = pd.read_csv("output/prediction_addition.csv",index_col=0)
evi_df = evi_df.loc[pat_df.index]

for i in pat_df.index:
    if "Journal" in pat_df.loc[i, "texts"].keys():
        pat_df.loc[i, '期刊'] = pat_df.loc[i, "texts"]["Journal"]
    else:
        pat_df.loc[i, '期刊'] = None
pat_df['文章标题'] = [i["Title"] for i in pat_df["texts"]]
pat_df["pmid"] = ["https://pubmed.ncbi.nlm.nih.gov/%s/" % i for i in pat_df["pmid"]]
pat_df["标注方式"] = "模型标注"

mac_df = pd.concat((pat_df,evi_df), axis=1)
mac_df.to_csv("output/machine_label.csv", encoding="utf_8_sig")

# 合并人工标注数据

In [4]:
# 读取人工标注文件
hum_df = pd.read_excel("input/human_label_1.0.xlsx", index_col=0, header=0)
hum_df.index = [i.lower() for i in hum_df.index]
hum_df["物种名称"] = [hum_df.loc[i,"物种名称"].lower() for i in hum_df.index]

hum_df = hum_df.loc[hum_df["是否是致病菌"]=="yes", :]
hum_df = hum_df[['物种名称', '本文中是肺部病原的概率','样本量', '年龄', '患者性别',  '免疫状态', '证据等级', '标注方式', '原文链接' ]]
hum_df['本文中是肺部病原的概率'] = hum_df['本文中是肺部病原的概率'].round(2)
hum_df["物种类别"] = "Bactology"
# 机器标注的
pat_df = pd.read_csv("output/machine_label.csv", index_col=0)
pat_df = pat_df[['spec_type', 'species', 'predition', 'num','age', 'gender', 'immu', 'evi', '标注方式', 'pmid']]
pat_df.columns = ['物种类别', '物种名称', '本文中是肺部病原的概率','样本量', '年龄',  '患者性别',  '免疫状态', '证据等级', '标注方式', '原文链接' ]
pat_df['本文中是肺部病原的概率'] = pat_df['本文中是肺部病原的概率'].round(2)
pat_df = pat_df.fillna("none")
# pat_df["发表年份"] = pat_df["发表年份"].astype(int)
# pat_df.loc[pat_df["发表年份"]==0, "发表年份"] = "none"
# for i in hum_df.index:
#     if i in pat_df.index:
#         hum_df.loc[i, "期刊"] = pat_df.loc[i,"期刊"]
#     pat_df.loc[i] = hum_df.loc[i]
dul_index = [i for i in set(hum_df.index) if i in set(pat_df.index)]
pat_df.loc[dul_index] = hum_df.loc[dul_index]
uniq_index = [i for i in hum_df.index if i not in dul_index]
all_df = pd.concat((hum_df.loc[uniq_index],pat_df),axis=0)
all_df.to_csv("output/all_label.csv",encoding='utf-8-sig')